In [ ]:
# Work06
## Loading random pseudo points location generated from QGIS
## Saving Google Street View (GSV) images according to their coordinates
## Setting id to identify each pseudo-absence point location
## Saving all GSV into a list for subsequent analysis
## Running Segment Anything Model (SAM) for image segmentation
## Applying written functions to filter out overlapping subset masks

In [1]:
# loading libraries
# pip install google_streetview
# pip install pyproj
import torch
torch.cuda.empty_cache() 
import google_streetview
import google_streetview.api
import google_streetview.helpers
import glob
import shapely
from shapely.geometry import Polygon
import shapely.plotting
import os 
os.environ['MPLCONFIGDIR'] = os.getcwd() + "/configs/"
import matplotlib
from matplotlib import pyplot as plt
import numpy as np
import cv2
import urllib.request
from PIL import Image
import pandas as pd
import pyproj

In [ ]:
# loading written functions
from mtp_function_yl import *

In [7]:
# preparing SAM
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)

    img = np.ones((sorted_anns[0]['segmentation'].shape[0], sorted_anns[0]['segmentation'].shape[1], 4))
    img[:,:,3] = 0
    for ann in sorted_anns:
        m = ann['segmentation']
        color_mask = np.concatenate([np.random.random(3), [0.35]])
        img[m] = color_mask
    ax.imshow(img)

from segment_anything import SamPredictor, sam_model_registry
sam = sam_model_registry["vit_h"](checkpoint="/home/yelhe/data/sam/sam_vit_h_4b8939.pth")
from segment_anything import SamAutomaticMaskGenerator, sam_model_registry
mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
# loading random psuedo-absence points
rp_koord = pd.read_csv('/home/yelhe/data/Acc/rpseudo_new.csv')
# transforming locations into coordinates
rp_koord = rp_koord[['x','y']]
rp_koord = rp_koord.assign(lat = None, lon = None)
transformer = pyproj.Transformer.from_crs("EPSG:2056", "EPSG:4326")
# a latitude; b longitude
a, b = transformer.transform(rp_koord[['x']], rp_koord[['y']])
rp_koord = rp_koord.assign(lon = b, lat = a)

In [ ]:
# saving GSV images
rp_koord = rp_koord.assign(url1 = None, url2 = None, url3 = None, url4 = None)

rp_koord.lon.astype(str)

# parameters setting
# heading 0/90/180/270
# fov 120 (in both horizontal and vertical dimensions)
# pitch 0

# using 'Google API key'
rp_koord['url1'] = 'https://maps.googleapis.com/maps/api/streetview?key='+ 'Google API Key'+ '&location=' + rp_koord.lat.astype(str) + '%2C+' + rp_koord.lon.astype(str) + '&heading=0&size=640x640&fov=120&pitch=0'
rp_koord['url2'] = 'https://maps.googleapis.com/maps/api/streetview?key='+ 'Google API Key'+ '&location=' + rp_koord.lat.astype(str) + '%2C+' + rp_koord.lon.astype(str) + '&heading=90&size=640x640&fov=120&pitch=0'
rp_koord['url3'] = 'https://maps.googleapis.com/maps/api/streetview?key='+ 'Google API Key'+ '&location=' + rp_koord.lat.astype(str) + '%2C+' + rp_koord.lon.astype(str) + '&heading=180&size=640x640&fov=120&pitch=0'
rp_koord['url4'] = 'https://maps.googleapis.com/maps/api/streetview?key='+ 'Google API Key'+ '&location=' + rp_koord.lat.astype(str) + '%2C+' + rp_koord.lon.astype(str) + '&heading=270&size=640x640&fov=120&pitch=0'

In [20]:
# setting id to identify each random pseudo-absence point location
rp_koord['id'] = rp_koord.index
rp_koord['id'] = rp_koord.id.astype(str)
rp_koord['id'] = rp_koord['id'].str.zfill(4)
# saving coordinates and id of each random pseudo-absence point location into a file
# rp_koord.to_csv('/home/yelhe/script/mt/output/rp_koord_new.csv',index = False)

In [26]:
# setting names for images using 'id' for random pseudo locations
name1 = rp_koord.id.astype(str) + '1'
name2 = rp_koord.id.astype(str) + '2'
name3 = rp_koord.id.astype(str) + '3'
name4 = rp_koord.id.astype(str) + '4'

In [1]:
# obtaining GSV images
for i in range(0,995):
    urllib.request.urlretrieve(rp_koord.url1[i], f'/home/yelhe/data/gsv_rp_new/{name1[i]}.jpg')
    urllib.request.urlretrieve(rp_koord.url2[i], f'/home/yelhe/data/gsv_rp_new/{name2[i]}.jpg')
    urllib.request.urlretrieve(rp_koord.url3[i], f'/home/yelhe/data/gsv_rp_new/{name3[i]}.jpg')
    urllib.request.urlretrieve(rp_koord.url4[i], f'/home/yelhe/data/gsv_rp_new/{name4[i]}.jpg')
    print(i)

# importing images into a list
from pathlib import Path
images = [cv2.imread(file) for file in glob.glob("/home/yelhe/data/gsv/GSV_filtered/*.jpg")]

In [34]:
# preparing for image segmentation 
gsv_imager1 = []
gsv_imager2 = []
gsv_imager3 = []
gsv_imager4 = []

nrf1 = []
nrf2 = []
nrf3 = []
nrf4 = []

# for i in range(0,995):
#     path1 = Path('/home/yelhe/data/gsv_rp_f/gsv_rp_nf/' + name1[i] + '.jpg')
#     c1 = path1.is_file()
#     path2 = Path('/home/yelhe/data/gsv_rp_f/gsv_rp_nf/' + name2[i] + '.jpg')
#     c2 = path2.is_file()
#     path3 = Path('/home/yelhe/data/gsv_rp_f/gsv_rp_nf/' + name3[i] + '.jpg')
#     c3 = path3.is_file()
#     path4 = Path('/home/yelhe/data/gsv_rp_f/gsv_rp_nf/' + name4[i] + '.jpg')
#     c4 = path4.is_file()
#     if c1 is True:
#         im1 = cv2.imread("/home/yelhe/data/gsv_rp_f/gsv_rp_nf/" + name1[i] + ".jpg")
#         gsv_imager1.append(im1)
#         nrf1.append(name1[i])
#     if c2 is True:
#         im2 = cv2.imread("/home/yelhe/data/gsv_rp_f/gsv_rp_nf/" + name2[i] + ".jpg")
#         gsv_imager2.append(im2)
#         nrf2.append(name2[i])
#     if c3 is True:
#         im3 = cv2.imread("/home/yelhe/data/gsv_rp_f/gsv_rp_nf/" + name3[i] + ".jpg")
#         gsv_imager3.append(im3)
#         nrf3.append(name3[i])
#     if c4 is True:
#         im4 = cv2.imread("/home/yelhe/data/gsv_rp_f/gsv_rp_nf/" + name4[i] + ".jpg")
#         gsv_imager4.append(im4)
#         nrf4.append(name4[i])

# preparing functions for image segmentation and mask output after SAM
def convert_mask(masks):
    mb01 = []
    mb02 = []
    mb = []
    n = len(masks)
    for i in range(0, n):
        mb01.append(masks[i]['segmentation'])
    for i in range(0, n):
        mb02.append(mb01[i]*1)
    for i in range(0, n):
        mb.append(mb02[i].astype('uint8'))
    return mb


def output_mask(im):
    im1 = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    mask_r = mask_generator.generate(im1)
    mask_c = convert_mask(mask_r)
    return mask_c  

In [2]:
# running SAM and output mask
# for i in range(0, 995):
#     m1 = gsv_imager1[i]
#     om1 = output_mask(m1)
#     np.save('/home/yelhe/scratch/gsvrp_output/' + nrf1[i] + '.npy', om1, allow_pickle=True)
#     m2 = gsv_imager2[i]
#     om2 = output_mask(m2)
#     np.save('/home/yelhe/scratch/gsvrp_output/' + nrf2[i] + '.npy', om2, allow_pickle=True)
#     m3 = gsv_imager3[i]
#     om3 = output_mask(m3)
#     np.save('/home/yelhe/scratch/gsvrp_output/' + nrf3[i] + '.npy', om3, allow_pickle=True)
#     m4 = gsv_imager4[i]
#     om4 = output_mask(m4)
#     np.save('/home/yelhe/scratch/gsvrp_output/' + nrf4[i] + '.npy', om4, allow_pickle=True)
#     print(i)

In [3]:
# loading all generated mask files
mask1 = []
mask2 = []
mask3 = []
mask4 = []
# for i in range(0,995):
#     m1 = np.load('/home/yelhe/scratch/gsvrp_output/' + nrf1[i] + '.npy', allow_pickle=True)
#     mask1.append(m1)
#     m2 = np.load('/home/yelhe/scratch/gsvrp_output/' + nrf2[i] + '.npy', allow_pickle=True)
#     mask2.append(m2)
#     m3 = np.load('/home/yelhe/scratch/gsvrp_output/' + nrf3[i] + '.npy', allow_pickle=True)
#     mask3.append(m3)
#     m4 = np.load('/home/yelhe/scratch/gsvrp_output/' + nrf4[i] + '.npy', allow_pickle=True)
#     mask4.append(m4)

In [ ]:
# running the filtering function and save filtered mask files
# for i in range(0, 995):
#     m1 = mask1[i]
#     om1 = filter_overlap(m1)
#     np.save('/home/yelhe/scratch/gsv_rp_output_filtered/' + n1[i] + 'f.npy', om1, allow_pickle=True)
#     m2 = mask2[i]
#     om2 = filter_overlap(m2)
#     np.save('/home/yelhe/scratch/gsv_rp_output_filtered/' + n2[i] + 'f.npy', om2, allow_pickle=True)
#     m3 = mask3[i]
#     om3 = filter_overlap(m3)
#     np.save('/home/yelhe/scratch/gsv_rp_output_filtered/' + n3[i] + 'f.npy', om3, allow_pickle=True)
#     m4 = mask4[i]
#     om4 = filter_overlap(m4)
#     np.save('/home/yelhe/scratch/gsv_rp_output_filtered/' + n4[i] + 'f.npy', om4, allow_pickle=True)
#     print(i)